In [50]:
import pandas as pd
import dask.dataframe as dd
seq_df_subset = pd.read_csv('../go_annotation/cafa3/cafa3_annotated_targets.csv.gz')

In [79]:
swissprot_data = pd.read_parquet('/projects/bpms/pstjohn/swissprot/parsed_swissprot_uniref_clusters.parquet')
missing = seq_df_subset[~seq_df_subset.sequence.isin(swissprot_data.sequence)]

In [53]:
from dask.diagnostics import ProgressBar

In [54]:
uniref100 = dd.read_parquet('/scratch/pstjohn/uniparc/uniref100/*', engine='pyarrow')

In [58]:
import os

In [62]:
from tqdm import tqdm

In [ ]:
results = []

for file in tqdm(os.listdir('/scratch/pstjohn/uniparc/uniref100/')):
    df = pd.read_parquet(os.path.join('/scratch/pstjohn/uniparc/uniref100/', file), engine='pyarrow')
    results += [missing.merge(df, right_on='Sequence', left_on='sequence')]

 43%|████▎     | 925/2136 [06:12<09:10,  2.20it/s]

In [70]:
matches = pd.concat(results)

In [132]:
swissprot_deduped = swissprot_data.drop_duplicates(subset='sequence')

In [133]:
seq_df_subset.shape

(3328, 3)

In [139]:
sp_merged = seq_df_subset.merge(swissprot_deduped[['UniRef100 ID', 'UniRef90 ID', 'UniRef50 ID', 'sequence', 'accession']],
                                left_on='sequence', right_on='sequence', how='left')

In [135]:
sp_merged.shape

(3328, 7)

In [145]:
to_add = matches.set_index('CAFA3ID').reindex(sp_merged.CAFA3ID)[['UniRef100 ID', 'UniRef90 ID', 'UniRef50 ID']].dropna()

sp_merged = sp_merged.set_index('CAFA3ID')
sp_merged.loc[to_add.index, ['UniRef100 ID', 'UniRef90 ID', 'UniRef50 ID']] = to_add
sp_merged = sp_merged.reset_index()

sp_merged.loc[1831, 'UniRef50 ID'] = 'UniRef50_P06968'
sp_merged.loc[1961, 'UniRef50 ID'] = 'UniRef50_B7NDV3'
sp_merged.loc[2785, 'UniRef50 ID'] = 'UniRef50_P28738'

In [146]:
sp_merged.shape

(3328, 7)

In [151]:
sp_merged
sp_merged.to_csv('../go_annotation/cafa3/cafa3_annotated_targets_with_swissprot.csv.gz', compression='gzip')